### The dataset for multi-label text classification was introduced in this kaggle challenge
- https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/


In [4]:
# !wget https://www.dropbox.com/s/6dygg8esbdywogl/jigsaw-toxic-comment-classification-challenge.zip

In [5]:
# !unzip jigsaw-toxic-comment-classification-challenge.zip

In [1]:
# utils
import torch
import pandas as pd

# data
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset

# model
import torch.nn as nn
import torch.nn.functional as F

# training
import torch.optim as optim
import tqdm

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda', index=0)

### Data Preparation

In [4]:
root = "./jigsaw-toxic-comment-classification-challenge/"

In [5]:
# create fields
TEXT = Field(lower=True, tokenize="spacy", batch_first=True)
LABEL = LabelField(batch_first=True, dtype=torch.float32)

In [6]:
# field and dataframe
train_fields = [
    ("id", None), 
    ("comment_text", TEXT), 
    ("toxic", LABEL), 
    ('severe_toxic', LABEL), 
    ('obscene', LABEL), 
    ('threat', LABEL),
    ('insult', LABEL),
    ('identity_hate', LABEL)
]

test_fields = [
    ("id", None), 
    ("comment_text", TEXT), 
]

In [7]:
# load the training dataset
data = TabularDataset(
    path="./jigsaw-toxic-comment-classification-challenge/train.csv",
    format="CSV",
    fields=train_fields,
    skip_header=True,
)

In [8]:
# split the total labelled data into train and val
train, val = data.split(split_ratio=0.98)

In [9]:
# load test data
test = TabularDataset(
    path="./jigsaw-toxic-comment-classification-challenge/test.csv",
    format="CSV",
    fields=test_fields,
    skip_header=True,
)

In [10]:
## Build the vocabulary
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [11]:
# train and val dataloader
BATCH_SIZE = 32
train_loader, val_loader = BucketIterator.splits(
    datasets=(train, val),
    batch_size=BATCH_SIZE,
    device=device,
    shuffle=True,
    sort=False
)

In [99]:
# test dataloader -> no specific reason to exclude it from above BucketIterator
test_loader = BucketIterator(
    dataset=test,
    batch_size=1,
    device=device,
    sort=False
)


In [12]:
print(len(train_loader), len(val_loader), len(test_loader))

4887 100 4787


In [13]:
def batch_wrapper(batch):
    """it actually combines multi-label into NxC tensor"""
    x = batch.comment_text
    y = torch.stack((batch.toxic, batch.severe_toxic, batch.obscene, batch.threat, batch.insult, batch.identity_hate), dim=1)
    y = y.type(torch.float)
    return x, y
    

In [14]:
batch = next(iter(train_loader))

In [15]:
x, y =  batch_wrapper(batch)

### Model

In [59]:
class Model(nn.Module):
    def __init__(self, hidden_dim, emb_dim=300, num_linear=1):
        super(Model, self).__init__() # don't forget to call this!
        self.embedding = nn.Embedding(len(TEXT.vocab), emb_dim)
        self.encoder = nn.LSTM(emb_dim, hidden_dim, num_layers=1)
        self.linear_layers = []
        for _ in range(num_linear - 1):
            self.linear_layers.append(nn.Linear(hidden_dim, hidden_dim))
            self.linear_layers = nn.ModuleList(self.linear_layers)
        self.predictor = nn.Linear(hidden_dim, 6)

    def forward(self, seq):
        hdn, _ = self.encoder(self.embedding(seq))
        feature = hdn[:, -1, :]
        for layer in self.linear_layers:
          feature = layer(feature)
        preds = self.predictor(feature)
        return preds

### Trianing 

In [88]:
def eval(model, data, criterion):
    losses = []
    with torch.no_grad():
        for batch in data:
            x, y = batch_wrapper(batch)
            outputs = model(x)
            loss = criterion(outputs, y.type_as(outputs))
            losses.append(loss.item())
    return sum(losses)/len(losses)

In [112]:
# config
EPOCHS = 10
LR = 1e-2
TOTAL_STEPS = len(train_loader)

In [ ]:
em_sz = 100
nh = 500
nl = 3
model = Model(nh, emb_dim=em_sz).to(device)

In [90]:
# criterion and optimizer
criterion  = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params=model.parameters(), lr=LR)

In [113]:
epoch_progress = tqdm.tqdm(total=EPOCHS, desc="EPOCH", position=0)
steps = 0

for epoch in range(EPOCHS):
    epoch_losses = []
    step_progress = tqdm.tqdm(total=len(train_loader), desc="EPOCH", position=0)
    for batch in train_loader:
        x, y = batch_wrapper(batch)
        y_ = model(x)
        
        optimizer.zero_grad()

        loss = criterion(y_, y)

        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

        if steps%1000==0:
          print(f'Step {steps}/{TOTAL_STEPS} | Train_loss {loss.item():.4f}')

        steps += 1
        step_progress.update(1)

    train_loss = sum(epoch_losses)/len(epoch_losses)
    val_loss = eval(model, val_loader, criterion)

    print(f'Epoch {epoch}/{EPOCHS} | Train_loss {train_loss:.4f} | Val_loss {val_loss:.4f}')
    
    epoch_progress.update(1)

EPOCH:   0%|          | 3/4887 [00:00<27:21,  2.98it/s]

Step 0/4887 | Train_loss 0.1710


EPOCH:  21%|██        | 1002/4887 [01:41<07:46,  8.33it/s]

Step 1000/4887 | Train_loss 0.0914


EPOCH:  41%|████      | 2002/4887 [03:23<04:57,  9.70it/s]

...

Step 47000/4887 | Train_loss 0.0855


EPOCH:  82%|████████▏ | 4019/4887 [06:50<01:28,  9.76it/s]

Step 48000/4887 | Train_loss 0.1897


EPOCH: 100%|██████████| 10/10 [1:23:22<00:00, 499.66s/it]

Epoch 9/10 | Train_loss 0.1478 | Val_loss 0.1443
